### Read in Data Frame

In [ ]:
import pandas as pd # need to import the pandas class, these are similar to packages in R

df = pd.read_excel('Demo Data/Calls and Sales per Agent.xlsx', index=False) # see the use of `pd` at the beginning, the phrase 'read_excel' is specific to pandas

df.head(5) # 'head' is a pandas term as well. notice how it is the color blue, the number is the amount of rows to output

### Add a new column
We will create a column that is called 'Close Rate' which is Sales/Calls

In [ ]:
# basic 
df['Close Rate'] = df['Sales']/df['Calls'] 

# intermediate - adding round function, getting percentage value by multiplying by 100
# df['Close Rate'] = round((df['Sales']/df['Calls'])*100,2) 

# advanced - adding a '%' to the number using 'map' 
# df['Close Rate'] = round(df['Sales']/df['Calls']*100,2).map("{:,.2f}%".format)

df.head(2)

### Rename a column in Pandas
We will use this template `df = df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})` and rename 'Agent ID' to 'agent_id'. 

In [ ]:
df = df.rename(columns={'Agent ID': 'agent_id'}) # notice how 'rename' is blue, it's a pandas keyword

df.head(3)

### Apply a function to Pandas Data Frame
Doing row level calculations depending on different columns in the data frame. This is comparable to 'Case Statements' in SQL.

Let's bucket the differet agents by their close Rate percentage.

In [ ]:
# create a funciton
def close_rate_bucket(row):
    if row['Close Rate'] >0 and  row['Close Rate'] < .05:
        val = 'Poor'
    elif row['Close Rate'] >= .05 and row['Close Rate'] < .1:
        val = 'Okay'
    elif row['Close Rate'] >= .1  and row['Close Rate'] < .15:
        val = 'Average'
    elif row['Close Rate'] >= .15  and row['Close Rate'] < .2:
        val = 'Good'
    elif row['Close Rate'] >= .2  and row['Close Rate'] < .25:
        val = "Great"
    else:
        val = "Other"
    return val

# apply the function to the pandas data frame 
df['close_rate_agent_buckets'] = df.apply(close_rate_bucket, axis=1) # where 'agent_buckets' is the new column name

#output
df.head(7)

### 'Case Statement' example in Pandas Second way
Another way to conduct a Case-Statement-esque in pandas is using `.loc`.  Add a new column called 'agent_call_buckets'.

In [ ]:
# set the THEN first
df['agent_call_buckets'] = 'Other' 

# implement your logic
df.loc[(df['Calls'] > 0) & (df['Calls'] < 50), 'agent_call_buckets'] = 'Low'
df.loc[(df['Calls'] >= 50) & (df['Calls'] < 100), 'agent_call_buckets'] = 'Medium'
df.loc[(df['Calls'] >= 100), 'agent_call_buckets'] = 'High'

# output
df.head(6)

### Performing a Group By
Let's group by 'agent_call_buckets' and 'close_rate_buckets' to see if we have any insights

In [ ]:
# create a subset, save it as a new data frame variable
group_by = df[['agent_id', 'agent_call_buckets', 'close_rate_agent_buckets']] # <- double [[]] means 'subset', so you can grab spefic columns

# perform the group by, notice how 'groupby' is blue, it's a pandas keyword
group_by = group_by.groupby(['agent_call_buckets', 'close_rate_agent_buckets']).count().reset_index()

# rename column
group_by = group_by.rename(columns={'agent_id': 'number_of_agents'})

group_by.head(9)

### Filter dataframe 
Let's filter our df dataframe to get all agents with 'Poor' close rate ratings. And then we will look at 'Great' Agents.

In [ ]:
poor_agents = df[df['close_rate_agent_buckets'] == 'Poor'] # <-- filter our dataframe to only agents with Poor close rates
poor_agents.head()

In [ ]:
great_agents = df[df['close_rate_agent_buckets'] == 'Great'] 
great_agents.head()

### Saving Data Frame 
We can save the different data frames we created into one Excel file with multiple sheets.

In [ ]:
import pandas as pd

#create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter('Output/Call Center Data.xlsx', engine='xlsxwriter')

#write each DataFrame to a specific sheet
group_by.to_excel(writer, sheet_name='Score Card', index=False)
poor_agents.to_excel(writer, sheet_name='Need Improvement Agents', index=False)
great_agents.to_excel(writer, sheet_name='Great Job Agents', index=False)
df.to_excel(writer, sheet_name="Raw Data", index=False)

#close the Pandas Excel writer and output the Excel file
writer.save()

### Overview
We have taken import, made our data manipulation in python, then exported to excel only using python. 